In [1]:
#IMPORT DES LIBRAIRIES

import os
from PIL import Image
import numpy as np
import gzip
import math

In [2]:
#INITIALISATION DES NOMS DE FICHIERS
input_dir = "1_Initial"
output_dir = "2_Compressed"

file_list = os.listdir(input_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
# PARCOURT DE CHAQUE FICHIER
for filename in file_list:
    #LECTURE DES BYTES
    with open(os.path.join(input_dir, filename), "rb") as file:
        file_bytes = file.read()
    
    #ON AJOUTE DES BYTES AU FICHIER POUR SAUVEGARDER LE NOM DU FICHIER ET MARQUER LA FIN DU FICHIER 
    formatted_file_bytes = filename.encode() + b'\x00' + file_bytes + b'\x00'
    
    #ON COMPRESS LES BYTES (LOSSLESS)
    compressed_bytes = gzip.compress(formatted_file_bytes)
    
    #ON AJOUTE DES BYTES POUR QUE LA TAILLE DE L'IMAGE SOIT UN CARRE
    length = len(compressed_bytes) / 4
    next_square = math.ceil(math.sqrt(length)) ** 2
    padding = int((next_square - length) * 4)
    compressed_bytes += b'\x00' * padding
    
    size_as_rgba = len(compressed_bytes) / 4 
    required_width_height = int(np.sqrt(size_as_rgba))
    
    #CONVERTION DES BYTES EN IMAGE PUIS SAUVEGARDE 
    data = np.frombuffer(compressed_bytes, dtype=np.uint8) 
    image = Image.frombytes('RGBA', (required_width_height, required_width_height), data)
    image.save(os.path.join(output_dir, filename.split(".")[0] + ".png"))